In [1]:
import polars as pl 
import numpy as np 
from tqdm import tqdm 
from datetime import datetime
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
from filterpy.kalman import KalmanFilter
from tqdm import tqdm
from filterpy.kalman import KalmanFilter
import numpy as np

In [16]:
root_folder = '/home/alqua/data/data_vdfs/'
df = pl.read_parquet(root_folder + "pressure_miso.par")

In [17]:
df

time,pump1_speed,pump4_speed,pump3_speed,outflow,pressure,level
"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64
2023-05-01 00:00:00 UTC,0.0,780.0,0.0,187.0,0.541,152.0
2023-05-01 00:00:01 UTC,0.0,780.0,0.0,180.0,0.564,152.0
2023-05-01 00:00:02 UTC,0.0,780.0,0.0,174.0,0.553,152.0
2023-05-01 00:00:03 UTC,0.0,780.0,0.0,168.0,0.561,152.0
2023-05-01 00:00:04 UTC,0.0,780.0,0.0,167.0,0.561,152.0
…,…,…,…,…,…,…
2024-02-16 11:27:02 UTC,1020.0,0.0,1020.0,751.0,0.648148,176.0
2024-02-16 11:27:03 UTC,1020.0,0.0,1020.0,730.0,0.648148,176.0
2024-02-16 11:27:04 UTC,1020.0,0.0,1020.0,720.0,0.651042,176.0


In [3]:
pump1_df = pl.read_parquet(root_folder + "pump1_power_siso.par")
pump2_df = pl.read_parquet(root_folder + "pump4_power_siso.par")
pump3_df = pl.read_parquet(root_folder + "pump3_power_siso.par")

In [4]:
df = df.join(pump1_df.drop(["pump1_speed"]), on="time").join(pump2_df.drop(["pump4_speed"]), on="time").join(pump3_df.drop(["pump3_speed"]), on="time")

In [5]:
t = 1
area = 18
initial_Qin = df[0, 4]
initial_Qout = df[0, 4]
initial_h = df[0, 6]
# Given error estimates
error_est_Qin = 1e-7
error_est_h = 0.1
error_est_Qout = 0.1

error_obs_h = 0.001
error_obs_Qout = 0.1

A = np.array([[1, 0, 0], [t/area, 1, -t/area], [0, 0, 1]])

# Define the initial state (replace initial_Qin, initial_h, initial_Qout with actual initial values)
initial_Qin = 0.0
initial_h = 0.0
initial_Qout = 0.0

# Initialize the Kalman Filter
kf = KalmanFilter(dim_x=3, dim_z=2)
kf.x = np.array([initial_Qin, initial_h, initial_Qout])  # Initial state
kf.F = A
kf.H = np.array([[0, 1, 0], [0, 0, 1]])  # Observation matrix

# Define the process noise covariance matrix (V)
kf.Q = np.diag([error_est_Qin, error_est_h, error_est_Qout])

# Define the observation noise covariance matrix (W)
kf.R = np.diag([error_obs_h, error_obs_Qout])

# Simulate the process
progress_bar = tqdm(total=len(df), desc="Processing")
height_kf = []
inflow_kf = []
outflow_kf = []

for index in range(1, df.shape[0]):
    kf.predict()
    measurement = np.array([df[index, 6], df[index, 4]])
    kf.update(measurement)
    estimates = kf.x.copy()
    inflow_kf.append(estimates[0])
    height_kf.append(estimates[1])
    outflow_kf.append(estimates[2])
    progress_bar.update(1)  # Update the progress bar

progress_bar.close()  # Close the progress bar when done


Processing: 100%|█████████▉| 15109617/15109618 [07:31<00:00, 33476.01it/s] 


In [6]:
data = {"inflow_kf": inflow_kf, "height_kf": height_kf, "outflow_kf": outflow_kf}

In [7]:
df = df[1:]
df = df.with_columns([
    pl.Series('inflow_kf', inflow_kf),
])

In [8]:
xdf = df.to_pandas()

In [13]:
xdf

,time,pump1_speed,pump4_speed,pump3_speed,outflow,pressure,level,pump1_power,pump4_power,pump3_power,inflow_kf
0,2023-05-01 00:00:01+00:00,0.0,780.0,0.0,180.0,0.564000,152.0,0.000000,29.500000,0.000000,8.063915
1,2023-05-01 00:00:02+00:00,0.0,780.0,0.0,174.0,0.553000,152.0,0.000000,31.200001,0.000000,12.808731
2,2023-05-01 00:00:03+00:00,0.0,780.0,0.0,168.0,0.561000,152.0,0.000000,30.700001,0.000000,17.345920
3,2023-05-01 00:00:04+00:00,0.0,780.0,0.0,167.0,0.561000,152.0,0.000000,32.400002,0.000000,21.588317
4,2023-05-01 00:00:05+00:00,0.0,780.0,0.0,165.0,0.564000,152.0,0.000000,32.799999,0.000000,25.556618
...,...,...,...,...,...,...,...,...,...,...,...
15109612,2024-02-16 11:27:02+00:00,1020.0,0.0,1020.0,751.0,0.648148,176.0,47.529999,0.000000,39.299999,703.532605
15109613,2024-02-16 11:27:03+00:00,1020.0,0.0,1020.0,730.0,0.648148,176.0,46.919998,0.000000,38.360001,703.534726
15109614,2024-02-16 11:27:04+00:00,1020.0,0.0,1020.0,720.0,0.651042,176.0,49.200001,0.000000,38.910000,703.536237
15109615,2024-02-16 11:27:05+00:00,1020.0,0.0,1020.0,717.0,0.653935,176.0,47.230000,0.000000,39.330002,703.537317


In [14]:
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
 
fig = FigureWidgetResampler(go.Figure())
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.add_trace(go.Scattergl(name=r'Inflow', showlegend=True), hf_x=xdf.index, hf_y=xdf['inflow_kf'])
fig.update_layout(height=400, template="plotly_dark")
display(fig)

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> Inflow <i style="color:#fc9944">~15k</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '2cc32a09-6fd3-46ec-ba3b-b9b1b18f152f',
              'x': array([       0,      495,    15850, ..., 15086558, 15094505, 15109616],
                         shape=(1000,), dtype=uint64),
              'y': array([  8.06391524, 180.01416656, 168.50228265, ..., 649.6339909 ,
                          664.33597621, 703.53827577], shape=(1000,))}],
    'layout': {'height': 400, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

In [15]:
df.write_parquet("inflow_kf_est.par")